# NBA 2024-25: Slump Shots & Recovery Shots
## Notebook 03: Slump & Recovery Detection
This notebook implements the core analytical logic of the study: identifying shooting slumps and recoveries at the player-game level.

Using rolling shot windows, each shot is classified into one of three states:
- neutral
- slump
- recovered

In [2]:
# Import libraries
import pandas as pd
import numpy as np
from tqdm import tqdm

In [3]:
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [4]:
# Display options
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 1000)
pd.set_option("display.width", 160)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.float_format", lambda x: f"{x:.2f}")

---
## Load Data

In [6]:
# Load cleaned shot-level data from 2024-25 regular season
shots = pd.read_parquet(r"C:\Users\dylan\OneDrive\Documents\Portfolio_Projects\NBA_2024_25_shot_level\03_python_outputs\nba_2024_25_shot_level_data_clean.parquet")

In [7]:
# Inspect structure
shots.head(10)

,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM,calc_dist
0,22400001,288,201143,Al Horford,1610612738,Boston Celtics,2,4,29,Jump Shot,3PT Field Goal,Above the Break 3,Left Side Center(LC),24+ ft.,26,-103,247,1,2024-11-12,BOS,ATL,267.62
1,22400001,299,201143,Al Horford,1610612738,Boston Celtics,2,3,47,Jump Shot,3PT Field Goal,Above the Break 3,Center(C),24+ ft.,29,-69,289,1,2024-11-12,BOS,ATL,297.12
2,22400001,372,201143,Al Horford,1610612738,Boston Celtics,3,10,41,Jump Shot,3PT Field Goal,Above the Break 3,Left Side Center(LC),24+ ft.,24,-172,180,0,2024-11-12,BOS,ATL,248.97
3,22400001,399,201143,Al Horford,1610612738,Boston Celtics,3,8,2,Jump Shot,3PT Field Goal,Above the Break 3,Right Side Center(RC),24+ ft.,26,102,241,0,2024-11-12,BOS,ATL,261.70
4,22400001,409,201143,Al Horford,1610612738,Boston Celtics,3,6,43,Running Finger Roll Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,1,13,6,1,2024-11-12,BOS,ATL,14.32
5,22400001,217,201950,Jrue Holiday,1610612738,Boston Celtics,2,9,0,Pullup Jump shot,3PT Field Goal,Above the Break 3,Left Side Center(LC),24+ ft.,26,-94,249,0,2024-11-12,BOS,ATL,266.15
6,22400001,226,201950,Jrue Holiday,1610612738,Boston Celtics,2,8,4,Driving Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,2,25,13,1,2024-11-12,BOS,ATL,28.18
7,22400001,244,201950,Jrue Holiday,1610612738,Boston Celtics,2,6,50,Running Pull-Up Jump Shot,3PT Field Goal,Above the Break 3,Center(C),24+ ft.,27,-54,265,0,2024-11-12,BOS,ATL,270.45
8,22400001,364,201950,Jrue Holiday,1610612738,Boston Celtics,3,11,43,Floating Jump shot,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,6,36,49,1,2024-11-12,BOS,ATL,60.80
9,22400001,446,201950,Jrue Holiday,1610612738,Boston Celtics,3,4,13,Pullup Jump shot,3PT Field Goal,Above the Break 3,Right Side Center(RC),24+ ft.,24,129,210,0,2024-11-12,BOS,ATL,246.46


In [8]:
# Inspect columns
shots.columns

Index(['GAME_ID', 'GAME_EVENT_ID', 'PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_NAME', 'PERIOD', 'MINUTES_REMAINING', 'SECONDS_REMAINING', 'ACTION_TYPE',
       'SHOT_TYPE', 'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_DISTANCE', 'LOC_X', 'LOC_Y', 'SHOT_MADE_FLAG', 'GAME_DATE', 'HTM', 'VTM',
       'calc_dist'],
      dtype='object')

In [9]:
# Number of unique players
shots["PLAYER_NAME"].nunique()

566

---
## Slump & Recovery Detection

In [11]:
# Create shot index (shot 1, shot 2, ..., shot n)
shots["shot_idx"] = shots.groupby(["GAME_ID", "PLAYER_ID"]).cumcount() + 1

In [12]:
# --- Detect "slump" and "recovery" (one player-game at a time) ---
def detect_slump_and_recovery(player_game_shots):
    """
    Labels each shot in a single player-game sequence as:
    - "neutral"   : default shooting state
    - "slump"     : shots taken while in a slump
    - "recovered" : the single shot that signals a recovery

    Rules:
    - A slump triggers after:
        • 0 made shots in the previous 3 attempts
    - All subsequent shots are slump shots until a recovery occurs.
    - A recovery triggers after:
        • 2 consecutive made shots, or
        • 2 made shots in the previous 3 attempts
    - In all other cases, the default state is "neutral"
    """
    df = player_game_shots.copy()   # each temporary DataFrame is one player's shots from one game
    df["phase"] = "neutral"   # default value
    
    shot_outcomes = df["SHOT_MADE_FLAG"].values
    n = len(df)   # total number of shots in the player-game
    phase_col = df.columns.get_loc("phase")

    state = "neutral"   # default state
    neutral_start_idx = 0   # current neutral streak begins
    recovery_window = []

    for i in range(n):   # i is the current shot
        # -----------------------------
        # IN NEUTRAL → CHECK FOR SLUMP
        # -----------------------------
        if state == "neutral":

            neutral_len = i - neutral_start_idx + 1   # how many shots in current neutral streak
            slump_triggered = False

            # Slump Condition A: 0-for-3
            if neutral_len >= 3:   # first 2 shots of every player-game will always be neutral
                if shot_outcomes[i-2:i+1].sum() == 0:   # previous 3 shots including current shot
                    slump_triggered = True

            if slump_triggered:
                df.iloc[i, phase_col] = "slump"
                state = "slump"
                recovery_window = []
            else:
                df.iloc[i, phase_col] = "neutral"

        # -----------------------------
        # IN SLUMP → CHECK FOR RECOVERY
        # -----------------------------
        else:
            df.iloc[i, phase_col] = "slump"

            # Track only last 3 shots
            recovery_window.append(shot_outcomes[i])   # append the current shot outcome to the recovery window
            if len(recovery_window) > 3:
                recovery_window.pop(0)   # only keep last 3 shots in recovery window  ("pop" the 0th index)

            # Recovery Condition A: 2-for-2
            if len(recovery_window) >= 2 and sum(recovery_window[-2:]) == 2:   # 2 makes in last 2 shots
                df.iloc[i, phase_col] = "recovered"
                state = "neutral"
                neutral_start_idx = i + 1   # the next shot returns to neutral state
                recovery_window = []

            # Recovery Condition B: 2-for-3
            if len(recovery_window) == 3 and sum(recovery_window) == 2:   # 2 makes in last 3 shots
                df.iloc[i, phase_col] = "recovered"
                state = "neutral"
                neutral_start_idx = i + 1   # the next shot returns to neutral state
                recovery_window = []

        # note: an "unrecovered" slump at the end of a game always resets to neutral at the start of a new game (i.e., slumps do not carry over game to game)
    return df

## 🔑
> This state logic is the most important cell of the study. Once implemented in the next cell, **every single shot attempt** will have a state (or phase) label now, from which analytical insights can be found (in Notebook 04).

In [14]:
# --- Apply this state logic to all player-game rows ---
tqdm.pandas(desc="Processing player-games")

shots = shots.groupby(["GAME_ID", "PLAYER_ID"], group_keys=False).progress_apply(detect_slump_and_recovery)

Processing player-games: 100%|██████████| 24835/24835 [01:21<00:00, 303.05it/s]


In [15]:
# --- Test one game from one player ---
test = shots[shots["PLAYER_NAME"] == "Jayson Tatum"]

test[["PLAYER_NAME", "SHOT_MADE_FLAG", "shot_idx", "phase"]].head(500)

,PLAYER_NAME,SHOT_MADE_FLAG,shot_idx,phase
53,Jayson Tatum,0,1,neutral
54,Jayson Tatum,0,2,neutral
55,Jayson Tatum,0,3,slump
56,Jayson Tatum,0,4,slump
57,Jayson Tatum,1,5,slump
58,Jayson Tatum,1,6,recovered
59,Jayson Tatum,0,7,neutral
60,Jayson Tatum,1,8,neutral
61,Jayson Tatum,0,9,neutral
62,Jayson Tatum,1,10,neutral


> To verify this state logic, I manually scanned through the first 500 shots from Jayson Tatum's season. All of the logic works as intended. The data is now ready for analysis.

---
## Save

In [18]:
# Save to parquet
shots.to_parquet("nba_2024_25_shot_level_data_final.parquet")